In [83]:
!git clone https://github.com/RubenFranc/CDA_Taller2

fatal: destination path 'CDA_Taller2' already exists and is not an empty directory.


In [84]:
!pip install keras-tuner

In [85]:
#Numpy y Pandas
import pandas as pd
import numpy as np
from numpy import asarray

#Librerías de Scikit Learn
import sklearn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

#Librerías de Keras y Tensor Flow
import tensorflow as tf
import keras_tuner as kt

from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Input, Rescaling, Lambda, MaxPooling2D, Conv2D, AveragePooling2D, Activation, Reshape
from keras import backend as K
from keras import applications
from keras.utils import plot_model
from tensorflow.keras.optimizers import Adam

#Gráficos
import matplotlib.pyplot as plt
%matplotlib inline

#Otras Librerías
import os
from PIL import Image
from tqdm import tqdm

#Verificamos las versiones de las librerías
print(f"La versión de sklearn es: {sklearn.__version__}")
print('Tensorflow version:', tf.__version__)

La versión de sklearn es: 1.5.2
Tensorflow version: 2.17.0


In [86]:
def convertImagesToArrays(dataset:str)->tuple:
    print('\nFORMATEANDO IMÁGENES DE '+dataset.upper())
    data='CDA_Taller2/model/data/'+dataset+'/Fruit'
    categories=os.listdir(data)
    im_size=50
    dataToArray=[]
    labels=[]
    for category in range(len(categories)):
        path=data+'/'+categories[category]
        label=category
        l=os.listdir(path)
        for im in tqdm(range(len(l))):
            new_im=Image.open(os.path.join(path,l[im])).resize((im_size,im_size))
            im_array=np.asanyarray(new_im)
            dataToArray.append(im_array)
            labels.append(label)

    num_images=len(dataToArray)
    index=np.int64(np.linspace(0,num_images-1,num_images))
    np.random.shuffle(index)
    s_data=[]
    s_labels=[]
    for i in range(len(index)):
        try:
            s_data.append(dataToArray[index[i]])
            s_labels.append(labels[index[i]])
        except Exception as e:
            pass
    s_dataToArray=np.array(s_data)
    s_labels=np.array(s_labels)
    return s_dataToArray,s_labels

In [87]:
s_train_data,s_train_labels=convertImagesToArrays('train')
s_test_data,s_test_labels=convertImagesToArrays('test')
s_val_data,s_val_labels=convertImagesToArrays('val')


FORMATEANDO IMÁGENES DE TRAIN


100%|██████████| 22/22 [00:00<00:00, 531.02it/s]



FORMATEANDO IMÁGENES DE TEST


100%|██████████| 22/22 [00:00<00:00, 314.69it/s]



FORMATEANDO IMÁGENES DE VAL


100%|██████████| 5/5 [00:00<00:00, 240.93it/s]


In [88]:
data='CDA_Taller2/model/data/train/Fruit'
categories=os.listdir(data)
categories

['Passion-Fruit',
 'Pomegranate',
 'Red-Grapefruit',
 'Mango',
 'Avocado',
 'Kiwi',
 'Peach',
 'Nectarine',
 'Apple',
 'Pear',
 'Melon',
 'Papaya',
 'Lemon',
 'Banana',
 'Satsumas',
 'Lime',
 'Pineapple',
 'Orange',
 'Plum']

In [89]:
model = Sequential(name="Mi_primer_RNC")

In [90]:
model.add(Input(shape=(256, 256, 3), batch_size=None, name='Capa_Entrada'))
model.add(Rescaling(1./255, name='Capa_Reescalado'))

model.summary()

Model: "Mi_primer_RNC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Capa_Reescalado (Rescaling)          │ (None, 256, 256, 3)         │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [91]:
model.add(Conv2D(64, (3, 3), activation='relu', name='Capa_Convolucional_1'))
model.add(MaxPooling2D((2, 2), name='Max_Pool_1'))
model.add(Conv2D(128, (3, 3), activation='relu', name='Capa_Convolucional_2'))
model.add(AveragePooling2D((2, 2), name='Avg_Pool_1'))
model.add(Conv2D(256, (3, 3), activation='relu', name='Capa_Convolucional_3'))
model.summary()

Model: "Mi_primer_RNC"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Capa_Reescalado (Rescaling)          │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Capa_Convolucional_1 (Conv2D)        │ (None, 254, 254, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Max_Pool_1 (MaxPooling2D)            │ (None, 127, 127, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Capa_Convolucional_2 (Conv2D)        │ (None, 125, 125, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Avg_Pool_1 (AveragePooling2D)        │ (None, 62, 62, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Capa_Convolucional_3 (Conv2D)        │ (None, 60, 60, 256)         │         295,168 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 370,816 (1.41 MB)

 Trainable params: 370,816 (1.41 MB)

 Non-trainable params: 0 (0.00 B)

In [93]:
model.add(Flatten(name='Flatten_Layer'))
model.add(Dense(128,activation='relu',name='Capa_Densa_1'))
model.add(Dense(64,activation='relu',name='Capa_Densa_2'))
model.summary()

ValueError: All layers added to a Sequential model should have unique names. Name 'Flatten_Layer' is already the name of a layer in this model. Update the `name` argument to pass a unique name.

In [ ]:
model.add(Dense(len(categories), activation='softmax', name='Capa_Respuesta'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3)
]

In [ ]:
device_name = tf.test.gpu_device_name()
gpu_name = '/device:GPU:0'
if device_name != gpu_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'CDA_Taller2/model/data/train/Fruit',
  label_mode='categorical',
  seed=82,
  image_size=(256, 256),
  batch_size=32
)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  'CDA_Taller2/model/data/test/Fruit',
  label_mode='categorical',
  seed=82,
  image_size=(256, 256),
  batch_size=32
)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  'CDA_Taller2/model/data/val/Fruit',
  label_mode='categorical',
  seed=82,
  image_size=(256, 256),
  batch_size=32
)

In [ ]:
with tf.device(gpu_name):
  history = model.fit(train_ds,
          validation_data=test_ds,
          epochs=10
  )

In [ ]:
y_true = list()
y_pred = list()

for images, labels in train_ds:
    pred = model.predict(images)
    for i in range(len(pred)):
        y_true.append(labels[i].numpy())
        y_pred.append((pred[i] == pred[i].max()).astype(int))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
print(classification_report(y_true,y_pred,target_names=list(categories)))

In [ ]:
plt.figure(figsize=(10, 10))

df = train_ds.take(1)
for images, labels in df:
  pred = model.predict(images)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(categories[tf.argmax(pred[i]).numpy()])
    plt.axis("off")


In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['recall_1'], label='Training recall')
plt.plot(history.history['val_recall_1'], label='Validation recall')
plt.plot(history.history['precision_1'], label='Training precision')
plt.plot(history.history['val_precision_1'], label='Validation precision')
plt.title('Training and validation metrics')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
y_true = list()
y_pred = list()

for images, labels in test_ds:
    pred = model.predict(images)
    for i in range(len(pred)):
        y_true.append(labels[i].numpy())
        y_pred.append((pred[i] == pred[i].max()).astype(int))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
print(classification_report(y_true, y_pred, target_names=list(categories)))

In [ ]:
plt.figure(figsize=(10, 10))

df = test_ds.take(1)
for images, labels in df:
  pred = model.predict(images)
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(categories[tf.argmax(pred[i]).numpy()])
    plt.axis("off")

In [ ]:
def construirMejorModelo(hip):
  bModel = Sequential(name="Modelo_Simple")

  # Entrada
  bModel.add(Input(shape=(256, 256, 3), name='Entrada'))

  # Reescalado
  bModel.add(Rescaling(1./255, name='Reescalado'))

  # Capas Convolucionales y de Pooling
  bModel.add(Conv2D(filters=hip.Int('conv_filters', min_value=32, max_value=128, step=32),
                     kernel_size=hip.Choice('conv_kernel', values=[3, 5]),
                     activation='relu',
                     name='Convolucional_1'))

  bModel.add(MaxPooling2D((2, 2), name='MaxPooling_1'))

  bModel.add(Conv2D(filters=hip.Int('conv_filters_2', min_value=64, max_value=256, step=64),
                     kernel_size=hip.Choice('conv_kernel_2', values=[3, 5]),
                     activation='relu',
                     name='Convolucional_2'))

  bModel.add(MaxPooling2D((2, 2), name='MaxPooling_2'))

  bModel.add(Conv2D(filters=hip.Int('conv_filters_3', min_value=32, max_value=128, step=32),
                     kernel_size=hip.Choice('conv_kernel_3', values=[3, 5]),
                     activation='relu',
                     name='Convolucional_3'))

  bModel.add(MaxPooling2D((2, 2), name='MaxPooling_3'))

  # Capas Densas
  bModel.add(Flatten(name='Flatten'))
  bModel.add(Dense(units=hip.Int('dense_units', min_value=64, max_value=128, step=64),
                    activation='relu', name='Densa'))
  bModel.add(Dense(19, activation='softmax', name='Salida'))

  # Compilación del modelo
  bModel.compile(optimizer="adam", loss=tf.keras.losses.CategoricalCrossentropy(),
                   metrics=["accuracy"])

  return bModel

In [ ]:
tuner = kt.RandomSearch(
    construirMejorModelo,
    objective="val_accuracy",
    project_name="cls_fruta",
    seed=82,
    max_trials=10)

In [ ]:
with tf.device(gpu_name):
  tuner.search(train_ds, validation_data=test_ds, callbacks=my_callbacks, epochs=10)

In [ ]:
best_model=tuner.get_best_models(num_models=1)[0]
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

In [ ]:
best_model.save('best_model_cls_fruta.keras')
best_model.summary()

In [ ]:
y_true = list()
y_pred = list()

for images, labels in train_ds:
    pred = best_model.predict(images)
    for i in range(len(pred)):
        y_true.append(labels[i].numpy())
        y_pred.append((pred[i] > 0.5).astype(int))  # Convertir a 1 si la probabilidad es mayor a 0.5, 0 en caso contrario

y_true = np.array(y_true)
y_pred = np.array(y_pred)